In [2]:
from TexSoup import TexSoup
import shutil
import re
import os
import difflib
import pprint

#os.path.exists(os.path.join(os.getcwd(), 'src/pages/sta'))

In [3]:
coursetags = {
    'Thermo':'thermodynamics',
}

course = 'Thermo'

tex_files = list(sorted([file[:-4] for file in os.listdir(f'../public/{course}') if file[-4:] == '.tex' and (file != 'main.tex')]))

if not os.path.exists(f'../src/pages/{coursetags[course]}'):
    os.mkdir(f'../src/pages/{coursetags[course]}')

In [4]:
tex_files

['Complex Problems',
 'Cycles',
 'Entropy',
 'First Law',
 'Heat and Work',
 'Phase Diagrams',
 'Second Law',
 'Substances',
 'Thermodynamic Systems',
 'Thermodynamic devices']

In [11]:
replacements = {
    'section': 'Section',
    'subsection': 'SubSection',
    'subsubsection': 'SubSubSection',
    'blue': 'BlueText',
    'red': 'RedText',
    'textit': 'em',
    'textbf': 'strong'
    "caption", "em"
}

SyntaxError: ':' expected after dictionary key (636849005.py, line 9)

In [12]:
for filename in tex_files:
    required_modules = ['Image']

    tex_path = f"../public/{course}/{filename}.tex"

    astro_path = f"../src/pages/{coursetags[course]}/{filename.lower().replace(' ', '_')}.astro"


    with open(tex_path, 'r+') as file:
        raw = file.read()

        data = TexSoup(raw)

        #matches = re.sub(r'(?<=\n)(?!\s*\\)(.*?)(?<!\})(?=\n)', raw)

        raw = re.sub(r'(?<=\n)(?!\s*\\)(.*?)(?<!\})(?=\n)', lambda m: f"<p>{m.group(1)}</p>", raw, flags=re.DOTALL)

        for tex, html in replacements.items():
            
            matches = data.find_all(tex)
            
            for match in list(matches):
                if tex == 'blue':
                    print(str(match)[:6], filename)

                if len(list(match.contents)) != 0:
                    #print(str(match))
                    
                    #raw = raw.replace(r"\%s{%s}" % (tex, list(match.contents)[0]), f"<{html}>{list(match.contents)[0]}</{html}>")

                    raw = raw.replace(str(match).replace(r'\blue {', r'\blue{'), f"<{html}>{' '.join(list([str(e) for e in match.contents]))}</{html}>")

                    if html != 'em' and html != 'strong' and (html not in required_modules):
                        required_modules.append(html)
        
        # ADDITIONAL CLEANUP

        raw = raw.replace('<Section>', '<Section title="">')
        raw = raw.replace('<SubSection>', '<SubSection title="">')
        raw = raw.replace('<SubSubSection>', '<SubSubSection title="">')
        #print([e for e in data.subsection.descendants])
        

        

        raw = raw.replace(r'\noindent', '')
        enum = list(data.find_all('enumerate'))

        if len(enum): 
            required_modules.append('Enumerate')
            required_modules.append('Item')

        for l in enum:
            s = str(l)
            final_str = ''
            for c in l.children:
                final_str += f"\t<Item>{list(c.text)[0]}</Item>\n"
            
            final_str = '<Enumerate>\n'+final_str+'</Enumerate>\n'

            raw = raw.replace(s, final_str)

        item = list(data.find_all('itemize'))

        if len(item): 
            required_modules.append('Itemize')
            if 'Item' not in required_modules:
                required_modules.append('Item')


        for l in item:
            #print(l)
            s = str(l)
            final_str = ''
            for c in l.children:
                final_str += f"\t<Item>{list(c.text)[0]}</Item>\n"
            
            final_str = '<Itemize>\n'+final_str+'</Itemize>\n'

            raw = raw.replace(s, final_str)

        comments = re.findall('\n%(.*)\n', raw)
        
        for comment in comments:
            raw = raw.replace(f'%{comment}', f'<!-- {comment} -->')


        inline_equations = re.findall('\$([^$]*)\$', raw)
        
        if len(inline_equations): required_modules.append('InlineEquation')

        for eq in inline_equations:
            db_backslash = eq.replace('\\','\\\\')
            raw = raw.replace(f'${eq}$', f'<InlineEquation equation="{db_backslash}" />')

        patterns = [
            r'\\begin\{equation\}(.*?)\\end\{equation\}',
            r'\\begin\{equation\*\}(.*?)\\end\{equation\*\}'
            r'\\begin\{align\*\}(.*?)\\end\{align\*\}'
            ]

        # Replacement function
        def replace_equation(match):
            content = match.group(1).strip()
            # Extract label if present
            label_match = re.search(r'\\label\{(.*?)\}', content)
            label = label_match.group(1) if label_match else ""
            # Remove the label from the content
            content = re.sub(r'\\label\{.*?\}', '', content).strip()
            # Return the replacement string
            return f"<DisplayEquation equation=\"{content.replace('<p>', '').replace('</p>', '')}\" title=\"{label}\" />"

        # Perform the replacement
        for pattern in patterns:
            raw = re.sub(pattern, replace_equation, raw, flags=re.DOTALL)
        required_modules.append('DisplayEquation')

        display_equations = re.findall('\[(.*)\]', raw)

        for eq in display_equations:
            db_backslash = eq.replace('\\','\\\\')
            if r'\[%s]'% (eq) in raw and ('DisplayEquation' not in required_modules): required_modules.append('DisplayEquation')
            raw = raw.replace(r'\[%s]'% (eq), f'<DisplayEquation equation="{db_backslash}" />')

        imports = 'import Layout from "../../layouts/Layout.astro" \n'

        for comp in required_modules:
            imports += f'import {comp} from "../../components/{comp}.astro" \n' 

        figs = data.find_all('figure')

        for fig in figs:
            try:
                caption = ' '.join([str(e) for e in list(fig.caption.contents)[1:]])
                
                actual_text = caption[6:-1] if caption[0:5] == r'\blue' else caption
                
                inline_equations = re.findall('\$([^$]*)\$', actual_text)

                for eq in inline_equations:
                    db_backslash = eq.replace('\\','\\\\')
                    actual_text = actual_text.replace(f'${eq}$', f'<InlineEquation equation="{db_backslash}" />')

                if caption[0:5] == r'\blue':
                    actual_text = f'<BlueText>{actual_text}</BlueText>'

                try:
                    src = list(fig.includegraphics.text)[-1]
                except:
                    src=""
                    
                try:
                    width = list(fig.includegraphics.text)[0].split(',')[1].split('=')[1].replace('in', '')
                except:
                    width = '5' 

                #print(actual_text)

                raw = raw.replace(r'\end{figure}', f'<Image src="/{course}/{src}" width="{width}">{actual_text}</Image>', 1)
            except:
                raw = raw.replace(str(fig), "<!-- DRAWING HERE -->")
        

        figs = data.find_all('figure*')

        for fig in figs:
            caption = ' '.join([str(e) for e in list(fig.caption.contents)[1:]])
            
            actual_text = caption[6:-1] if caption[0:5] == r'\blue' else caption
            
            inline_equations = re.findall('\$([^$]*)\$', actual_text)

            for eq in inline_equations:
                db_backslash = eq.replace('\\','\\\\')
                actual_text = actual_text.replace(f'${eq}$', f'<InlineEquation equation="{db_backslash}" />')

            if caption[0:5] == r'\blue':
                actual_text = f'<BlueText>{actual_text}</BlueText>'

            try:
                src = list(fig.includegraphics.text)[-1]
            except:
                src=""

            try:
                width = list(fig.includegraphics.text)[0].split(',')[1].split('=')[1].replace('in', '')
            except:
                width = '5' 

            #print(actual_text)

            raw = raw.replace(r'\end{figure*}', f'<Image src="/{course}/{src}" width="{width}">{actual_text}</Image>', 1)

        #print(imports)

        raw = '---\n' + imports + '---\n' + f'<Layout title="{filename}">' + '\n' + raw + '\n' +'</Layout>'
        

        astro_file = open(astro_path, mode='w+')

        astro_file.write(raw)

        astro_file.close()



\blue{ Thermodynamic Systems
\blue{ Thermodynamic Systems
\blue{ Thermodynamic Systems


In [21]:
for filename in tex_files:
    required_modules = ['Image']

    tex_path = f"../public/{course}/{filename}.tex"

    astro_path = f"../src/pages/{coursetags[course]}/{filename.lower().replace(' ', '_')}.astro"


    with open(tex_path, 'r+') as file:
        raw = file.read()

        data = TexSoup(raw)

        for match in re.finditer(r'(?<=\n\n)(?!\\)(.*?)(?<!\})(?=\n\n)', raw):
            print(match)

<re.Match object; span=(790, 1265), match='The \\emph{state principle} is a guide to aid in >
<re.Match object; span=(1311, 1634), match='We next apply our knowledge of thermodynamic devi>
<re.Match object; span=(8887, 9708), match='During the cycle, the system contacts sequentiall>
<re.Match object; span=(10055, 10519), match='In processes $2 \\to 3$ and $3 \\to 4$, work is d>
<re.Match object; span=(21392, 21663), match='Combining the $p(V)$ relations for the ideal gas >
<re.Match object; span=(33592, 34221), match='Instead, the condenser stream is taken to the sat>
<re.Match object; span=(6215, 6215), match=''>
<re.Match object; span=(18823, 18823), match=''>
<re.Match object; span=(3844, 3932), match='%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%>
<re.Match object; span=(16351, 16426), match='\t\\draw[ultra thick, *-] (4.6,1) -- (5.6,2) node>
<re.Match object; span=(6684, 6686), match='\t\t'>
<re.Match object; span=(13633, 13922), match='The addition of energy to a system in it